In [54]:
import torch
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

<div class="jumbotron">
    <h1 class="display-1">自动微分</h1>
    <hr class="my-4">
    <p>主讲：李岩</p>
    <p>管理学院</p>
    <p>liyan@cumtb.edu.cn</p>
</div>

## 模型优化

- 在深度学习中，“训练”模型并不断更新模型，使它们在看到越来越多的数据时变得越来越好。

- 通常情况下，变得更好意味着**最小化**一个*损失函数*（loss function），即一个衡量“模型有多糟糕”这个问题的分数。

- 将拟合模型的任务分解为两个关键问题：
    - *优化*（optimization）：用模型拟合观测数据的过程
    - *泛化*（generalization）：数学原理和实践者的智慧，能够指导生成出有效性超出用于训练的数据集本身的模型

- 求导是几乎所有深度学习优化算法的关键步骤

神经网络的权重更新

$$
Q(\mathbf{w})=\sum_{i=1}^NQ_i(\mathbf{w})
$$

$$
\mathbf{w}_{t+1}=\mathbf{w}_t-\eta\sum_{i=1}^d\nabla_{\mathbf{w}}Q_i(\mathbf{w})
$$

<img src="../img/2_preliminaries/saddle_point_evaluation_optimizers.gif" width=80%>

- 深度学习框架通过自动计算导数，即*自动微分*（automatic differentiation）

- 根据设计好的模型，系统会构建一个*计算图*（computational graph），跟踪计算是哪些数据通过哪些操作组合起来产生输出

- 自动微分使系统能够随后反向传播梯度
    - *反向传播*（backpropagate）意味着跟踪整个计算图，填充关于每个参数的偏导数

### 计算机计算微分的方式

- 手动计算微分，再编写代码
- 利用Mathematica, Maple等进行符号计算（symbolic differentiation）
- 利用有限差分近似（finite difference approximations）的方法进行数值计算
- 自动微分

#### 手动计算

- 理论研究需要解析解

> In this article, <mark>a new objective function is defined  and both this function and tis gradient are derived in closed-from for surfaces and volumes.</mark> This method opens a wide range of possibilities...

- 进行数学证明与性质分析
- 但是，当只需要在优化的时候计算微分，并不需要这些推导过程

#### 符号计算

- 通过变量符号自动推导微分的表达式

- 与手动计算类似，有可能发现优化问题的解析解的性质
- 但是，对于计算微分值却是低效的，很容易导致“表达式膨胀”（expression swell）

例，对于logistic map:
$$
l_1 = x\\
l_{n+1}=4l_n(1-l_n)
$$
计算$l_n$关于$x$的导数

<center><img src="../img/2_preliminaries/expressionSwell.png" width=80%></center>

<center><img src="../img/2_preliminaries/expressionSwell2.png" width=80%></center>

#### 数值计算

- 对于多元函数$f:\mathbb{R}^n\to\mathbb{R}$，可以近似计算该函数的微分

$$
\frac{\partial f(\mathbf{x})}{\partial x_i}\approx\frac{f(\mathbf{x}+h\mathbf{e}_i)-f(\mathbf{x})}{h}
$$

其中，$\mathbf{e}_i$是第$i$个单位向量，$h$是一个很小的正数

- 对于$n$维向量，数值计算的时间复杂度是$O(n)$
- $h$的选择对结果影响较大

#### 自动微分

- 一种获取计算某个值的程序，并自动构建计算该值的导数过程的一般方法

## 计算图

- 计算图：反映函数计算依赖关系的有向图（Directed Acyclic Graph, DAG）

- 节点（node）：代表变量，可以是标量、向量、矩阵、张量等

- 节点也可以是一个操作（operation），即对一个或多个变量的简单函数

- 边（edge）：反映了输入到输出的关系

例，给出表达式$e=(a+b)*(b+1)$的计算图，其中$a=2,b=1$

引入两个中间变量：
$$
c=a+b\\
d=b+1
$$

<center><img src="../img/2_preliminaries/computationGraph1.png" width=80%></center>

在边上显示导数

<center><img src="../img/2_preliminaries/computationGraph2.png" width=80%></center>

- 可以得到节点之间的间接影响

### 自动微分计算的两种模式 

#### 正向模式

- 从计算图的一个输入节点开始，逐步移动到图的最终输出节点。在每个节点，将指向该节点的导数求和，表明该输入对该节点的总影响

- 跟踪一个输入节点如何影响计算图上的**每个**节点

用正向模式计算每个节点关于$b$的导数

<center><img src="../img/2_preliminaries/computationGraph3.png" width=80%></center>

#### 反向模式

- 从计算图的最终输出节点开始，逐步移动到图的输入节点。在每个节点，将从该节点发出的导数求和

- 跟踪计算图的每个节点如何影响**一个**最终输出节点

用反向模式计算$e$关于每个节点的导数

<center><img src="../img/2_preliminaries/computationGraph4.png" width=80%></center>

例，给出函数$f(x,y)=log(x*y)$的计算图

<center><img src="../img/2_preliminaries/computationalgraph.png" width=80%></center>

- 正向计算将每个基本运算的偏导数添加到计算图中
- 反向计算求出每个节点的偏导数

<center><img src="../img/2_preliminaries/extended_computational_graph.png" width=80%></center>

## 向量微分

- 向量微分的基础是***Jacobian Matrix***

- 给定函数 $\boldsymbol{f}: R^n\to R^m$

$$
\boldsymbol{f(x)}=[f_1(x_1,\cdots,x_n),f_2(x_1,\cdots,x_n),\cdots,f_m(x_1,\cdots,x_n)]
$$

- Jacobian矩阵为

$$
\frac{\partial{\boldsymbol{f}}}{\partial{\boldsymbol{x}}}=\begin{bmatrix}
\frac{\partial{f_1}}{\partial{x_1}} & \cdots & \frac{\partial{f_1}}{\partial{x_n}}\\
\vdots & \ddots & \vdots\\
\frac{\partial{f_m}}{\partial{x_1}} & \cdots & \frac{\partial{f_m}}{\partial{x_n}}
\end{bmatrix}
$$

例，有一个函数$\boldsymbol{f}(x)=[f_1(x),f_2(x)]$，另一个函数$\boldsymbol{g(y)}=[g_1(y_1,y_2),g_2(y_1,y_2)]$，二者的复合函数为$\boldsymbol{g}(x)=\left[g_1\left(f_1(x),f_2(x)\right),g_2\left(f_1(x),f_2(x)\right)\right]$,
则$\boldsymbol{g}$关于$x$的偏导数为？

$$
\frac{\partial \boldsymbol{g}}{\partial x}=\begin{bmatrix}
\frac{\partial}{\partial x}g_1\left(f_1(x),f_2(x)\right)\\
\frac{\partial}{\partial x}g_2\left(f_2(x),f_2(x)\right)
\end{bmatrix}=\begin{bmatrix}
\frac{\partial g_1}{\partial f_1}\frac{\partial f_1}{\partial x} +  \frac{\partial g_1}{\partial f_2}\frac{\partial f_2}{\partial x}\\
\frac{\partial g_2}{\partial f_1}\frac{\partial f_1}{\partial x} + \frac{\partial g_2}{\partial f_2}\frac{\partial f_2}{\partial x}
\end{bmatrix}
\label{eq:jacobianproduct}
$$

- 链式求导（式\eqref{eq:jacobianproduct}）是两个Jacobian矩阵的乘积

$$
\frac{\partial \boldsymbol{g}}{\partial x}=\frac{\partial \boldsymbol{g}}{\partial \boldsymbol{f}}\frac{\partial \boldsymbol{f}}{\partial x}=\begin{bmatrix}
\frac{\partial g_1}{\partial f_1} & \frac{\partial g_1}{\partial f_2}\\
\frac{\partial g_2}{\partial f_1} & \frac{\partial g_2}{\partial f_2}
\end{bmatrix} \begin{bmatrix}
\frac{\partial f_1}{\partial x}\\
\frac{\partial f_2}{\partial x}
\end{bmatrix}
$$

## 基本方法

### 例1

对函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于列向量$\mathbf{x}$求导

#### 符号计算

假设$\boldsymbol{x}=(x_1,x_2,\cdots,x_n)^T$，则

$$
y=2\boldsymbol{x}^T\boldsymbol{x}=2\sum_{i=1}^nx_i^2
$$

$$
\frac{\partial y}{\partial \boldsymbol{x}}=\left(\frac{\partial y}{\partial x_1},\frac{\partial y}{\partial x_2},\cdots,\frac{\partial y}{\partial x_n}\right)
$$

$$
\frac{\partial y}{\partial \boldsymbol{x}}=\left(4x_1,4x_2,\cdots,4x_n\right)
$$

#### torch计算

##### 生成向量$\boldsymbol{x}$

In [18]:
x = torch.arange(4.0)
f'向量x为{x}'

'向量x为tensor([0., 1., 2., 3.])'

- 在计算$\boldsymbol{y}$关于$\boldsymbol{x}$的梯度之前，需要一个地方来存储梯度

In [19]:
x.requires_grad_(True) # 开始在这个张量上记录自动微分的操作
f'x的梯度为{x.grad}'  # 张量的属性，记录微分结果

tensor([0., 1., 2., 3.], requires_grad=True)

'x的梯度为None'

- 也可以在生成向量的时候指定需要存储自动微分记录

```python
torch.arange(start=0, end, step=1, requires_grad=False)
```
- 令`requires_grad=True`即可

##### 正向计算$y$

In [20]:
y = 2 * torch.dot(x, x)
f'y的值为{y}'

'y的值为28.0'

##### 调用反向传播函数自动计算$\boldsymbol{y}$关于$\boldsymbol{x}$每个分量的梯度

In [21]:
y.backward()
f'x的梯度为{x.grad}'

'x的梯度为tensor([ 0.,  4.,  8., 12.])'

### 例2

计算$\boldsymbol{x}$的另一个函数$\boldsymbol{y}=\sum_{i=1}^nx_i$

#### 梯度置零

- 在默认情况下，PyTorch会累积梯度，因此需要清除之前存储的梯度值

In [22]:
x.grad.zero_()

tensor([0., 0., 0., 0.])

#### 正向计算$\boldsymbol{y}$

In [23]:
y = x.sum()
f'y的值为{y}'

'y的值为6.0'

#### 反向计算梯度

In [24]:
y.backward()
f'x的梯度为{x.grad}'

'x的梯度为tensor([1., 1., 1., 1.])'

### 向量（矩阵）对向量（矩阵）的梯度

- 当`y`不是标量时，向量$\boldsymbol{y}$关于向量$\boldsymbol{x}$的导数的最自然解释是一个矩阵

- 对非标量调用`backward`需要传入一个`gradient`参数，该参数可以认为是输出向量关于自身的梯度

- 实际上，`torch`关于向量微分计算的是一个**向量**和**Jacobian矩阵**的乘积

假设一个$m$维向量$\vec{y}$是$n$维向量$\vec{x}$的函数，即$\vec{y}=f(\vec{x})$，则$\vec{y}$关于$\vec{x}$的梯度由Jacobian矩阵表达，

$$
J=\begin{pmatrix}
\frac{\partial y_1}{\partial x_1},\cdots,\frac{\partial y_m}{\partial x_n}\\
\vdots,\ddots,\vdots\\
\frac{\partial y_m}{\partial x_1},\cdots,\frac{\partial y_m}{\partial x_n}
\end{pmatrix}
$$

- 给定一个向量$\vec{v}$，`torch`的自动微分计算的是$J^T\cdot\vec{v}$或者$\vec{v}^T\cdot J$

- 如果$\vec{v}$恰好是一个关于$\vec{y}$的标量函数，$l=g(\vec{y})$，的梯度，即
$$
\vec{v}=\begin{pmatrix}
\frac{\partial l}{\partial y_1},\cdots,\frac{\partial l}{\partial y_m}
\end{pmatrix}^T
$$

- 则，向量和Jacobian矩阵的乘积就是$l$关于向量$\vec{x}$的梯度
$$
\vec{v}^T\cdot J=\begin{pmatrix}
\frac{\partial l}{\partial y_1},\cdots,\frac{\partial l}{\partial y_m}
\end{pmatrix}\begin{pmatrix}
\frac{\partial y_1}{\partial x_1},\cdots,\frac{\partial y_m}{\partial x_n}\\
\vdots,\ddots,\vdots\\
\frac{\partial y_m}{\partial x_1},\cdots,\frac{\partial y_m}{\partial x_n}
\end{pmatrix}=\begin{pmatrix}
\frac{\partial l}{\partial x_1},\cdots,\frac{\partial l}{\partial x_n}
\end{pmatrix}
$$

- 向量$\vec{v}$就是`backward`函数用于计算向量对向量微分时候需要指定的参数`gradient`

例如，计算$\boldsymbol{y}=\boldsymbol{x}*\boldsymbol{x}$的偏导数

In [27]:
x.grad.zero_() # 向量x的偏导数置零

tensor([0., 0., 0., 0.])

In [51]:
y = x * x
f'y的值为{y}'

'y的值为tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)'

In [31]:
y.backward(gradient=torch.ones(len(y)))
f'关于x的偏导数为{x.grad}'

'关于x的偏导数为tensor([0., 2., 4., 6.])'

- 上述方式等同于下述方法

```python
y.sum.backward()
```

In [52]:
x.grad.zero_()
y.sum().backward()
x.grad

tensor([0., 0., 0., 0.])

tensor([0., 2., 4., 6.])

## 分离计算

- 将某些计算移动到记录的计算图之外
- 假设`y`是作为`x`的函数计算的，而`z`则是作为`y`和`x`的函数计算的。想象一下，我们想计算`z`关于`x`的梯度，但由于某种原因，希望将`y`视为一个常数，并且只考虑到`x`在`y`被计算后发挥的作用

- 使用`detach()`函数实现分离计算

In [39]:
x.grad.zero_()
y = x * x
f'y的值为{y}'

tensor([0., 0., 0., 0.])

'y的值为tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)'

In [40]:
u = y.detach() # 分离y获得新变量u,该变量与y具有相同的值，但丢弃计算图中如何计算y的任何信息
f'u的值为{u}'

'u的值为tensor([0., 1., 4., 9.])'

In [41]:
z = u * x  # 继续计算z

z.sum().backward()  # 获得z关于x的偏导
f'z关于x的偏导为{x.grad}'

'z关于x的偏导为tensor([0., 1., 4., 9.])'

In [42]:
x.grad == u  # 关于x的偏导是否与u相等？

tensor([True, True, True, True])

In [43]:
x.grad.zero_()
y.sum().backward()   # 计算y关于x的偏导
f'y关于x的偏导为{x.grad}'
x.grad == 2 * x      # 关于x的偏导是否与2x相等？

tensor([0., 0., 0., 0.])

'y关于x的偏导为tensor([0., 2., 4., 6.])'

tensor([True, True, True, True])

## 控制流的梯度

- 即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），仍然可以计算得到变量的梯度

In [44]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [48]:
a = torch.randn(size=(), requires_grad=True)
f'a为{a}'
d = f(a)
f'd为{d}'
d.backward()
f'd关于a的偏导为{a.grad}'

'a为-0.06573323160409927'

'd为-107697.328125'

'd关于a的偏导为1638400.0'

In [49]:
a.grad == d / a   # 由于f是关于a分段线性，因此可以用d/a检验自动微分计算是否正确

tensor(True)